### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [15]:


pd.DataFrame([purchase_data["SN"].nunique()], columns = ["Total Players"])


,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [57]:
# create a dataframe and find the number of unique items
data_file_df = pd.DataFrame([purchase_data["Item Name"].nunique()], columns = ["Number of Unique Items"])
#number_of_unique_items = purchase_data["Item Name"].nunique()], columns = ["Number of Unique Items"]

# find the average price
average = purchase_data["Price"].mean()
data_file_df["Average Price"] = average

# Calculate the number of purchases
number_of_purchases = purchase_data["Purchase ID"].count()
data_file_df["Number of Purchases"] = number_of_purchases

# Calucate the total revenues
total_revenues = purchase_data["Price"].sum()
data_file_df["Total Revenues"] = total_revenues


# set ALL float columns to '${:,.2f}' formatting (including the percentage)
# format_dict = {col_name: '${:,.2f}' for col_name in contribution.select_dtypes(float).columns}
# override the percentage column
# format_dict['Individual % of total'] = '{:.1%}'
# contribution.head().style.format(format_dict)


# format columns
format_dict = {col_name: '${:,.2f}' for col_name in data_file_df.select_dtypes(float).columns}

data_file_df.head().style.format(format_dict)

,Number of Unique Items,Average Price,Number of Purchases,Total Revenues
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [59]:
# Get a count of the males, females undisclosed
count = purchase_data["Gender"].value_counts()

# Create a dataframe and add the count
gender_df = pd.DataFrame({"Total Counts": (count)})

# Get a total gender count
total = purchase_data["Gender"].count()

# Find the percent of each gender
perc_gender_total = count/total

# Add the percents to the dataframe
gender_df["Percent of Players"] = perc_gender_total

# format columns
format_dict = {col_name: '{:.1%}' for col_name in gender_df.select_dtypes(float).columns}

gender_df.head().style.format(format_dict)

,Total Counts,Percent of Players
Male,652,83.6%
Female,113,14.5%
Other / Non-Disclosed,15,1.9%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
# Using GroupBy in order to separate the data into fields according to "Gender" values
grouped_gender_df = purchase_data.groupby(['Gender'])

# The object returned is a "GroupBy" object and cannot be viewed normally...
print(grouped_gender_df)

# In order to be visualized, a data function must be used...
grouped_gender_df.count().head()

,Purchase ID,SN,Age,Item ID,Item Name,Price
Gender,,,,,,
Female,113,113,113,113,113,113
Male,652,652,652,652,652,652
Other / Non-Disclosed,15,15,15,15,15,15


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
# Create the bins in which Data will be held
# Bins are 0, 59.9, 69.9, 79.9, 89.9, 100.   
bins = [0, 59.9, 69.9, 79.9, 89.9, 100]

# Create the names for the five bins
group_names = ["F", "D", "C", "B", "A"]

# put the values in the appropriate bins
df["Test Score Summary"] = pd.cut(df["Test Score"], bins, labels=group_names, include_lowest=True)
df

NameError: name 'df' is not defined

## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
# Create the bins in which Data will be held
# Bins are 0, 59.9, 69.9, 79.9, 89.9, 100.   
bins = [0, 59.9, 69.9, 79.9, 89.9, 100]

# Create the names for the five bins
group_names = ["F", "D", "C", "B", "A"]

# put the values in the appropriate bins
df["Test Score Summary"] = pd.cut(df["Test Score"], bins, labels=group_names, include_lowest=True)
df

NameError: name 'df' is not defined

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
# Using GroupBy in order to separate the data into fields according to "Gender" values
grouped_gender_df = purchase_data.groupby(['Gender'])

# The object returned is a "GroupBy" object and cannot be viewed normally...
print(grouped_gender_df)

# In order to be visualized, a data function must be used...
grouped_gender_df.count().head()

,Purchase ID,SN,Age,Item ID,Item Name,Price
Gender,,,,,,
Female,113,113,113,113,113,113
Male,652,652,652,652,652,652
Other / Non-Disclosed,15,15,15,15,15,15


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
# Using GroupBy in order to separate the data into fields according to "Gender" values
grouped_gender_df = purchase_data.groupby(['Gender'])

# The object returned is a "GroupBy" object and cannot be viewed normally...
print(grouped_gender_df)

# Sorting the DataFrame based on "Freedom" column
# Will sort from lowest to highest if no other parameter is passed
freedom_df = happiness_df.sort_values("Freedom")
freedom_df.head()

# In order to be visualized, a data function must be used...
grouped_gender_df.count().head()

NameError: name 'happiness_df' is not defined

## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
# Sorting the DataFrame based on "Freedom" column
# Will sort from lowest to highest if no other parameter is passed
freedom_df = happiness_df.sort_values("Freedom")
freedom_df.head()

NameError: name 'happiness_df' is not defined